In [1]:
import pandas as pd
import seaborn as sns
import math
import random

In [2]:
df = pd.read_csv('cars.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'cars.csv'

In [ ]:
testIdx = []
while len(testIdx) < 10:
    num = random.randint(0, len(df)-1)
    if not num in testIdx:
        if df.loc[num, "miles_driven"] < 80000:
            testIdx.append(num)

testIdx

In [ ]:
trainDF = df.drop(testIdx).reset_index(drop=True)
testDF = df.iloc[testIdx].reset_index(drop=True)

In [ ]:
#little to no correlation between age and sales_price, so age will be dismissed as a possible contributor
sns.scatterplot(data=trainDF, x='age', y='sales_price')

In [ ]:
sns.scatterplot(data=trainDF, x='miles_driven', y='sales_price')
sns.scatterplot(data=testDF, x='miles_driven', y='sales_price')

In [ ]:
print(str(min(df['miles_driven'])))

In [ ]:
#equasion to use : A(x-H)^2 + B
#A must be between -0.000002 and -0.000001
#B must be between 6750 and 7250
#H must be between 5750 and 5850

#randomly select values for A and B
def trainModel():
    A = round(random.uniform(-0.000002, -0.000001),10)
    B = round(random.uniform(6750, 7250),3)
    H = round(random.uniform(5750, 5850),3)
    return A, H, B

# A, H, B = trainModel()
# print("equation: " + str(A) + "(x - " + str(H) + ")^2 + " + str(B))

In [ ]:
def testModel(testDF, A, H, B):
    testNums = []
    for i in range(len(testDF)):
        actual = testDF.loc[i, 'sales_price']
        estimate = ((A) * (testDF.loc[i, 'miles_driven'] - H)**2 + B)
        testNums.append((actual - estimate)**2)
       
    return (sum(testNums) / len(testNums))**0.5

rmseMins = {'rmse':[], 'A':[], 'H':[], 'B':[]}
rmseMins = pd.DataFrame(rmseMins)
for i in range(50):
    rmseNums = {'rmse':[], 'A':[], 'H':[], 'B':[]}
    rmseNums = pd.DataFrame(rmseNums)
    for j in range(1000):
        A, H, B = trainModel()
        rmse = testModel(testDF, A, H, B)
        A = str(A)
        locDF = {'rmse': [rmse], 'A': [A], 'H':[H], 'B':[B]}
        locDF = pd.DataFrame(locDF)
        rmseNums = pd.concat([rmseNums, locDF], ignore_index=True)
        #print(rmse, A, H, B)
    minRMSE = min(rmseNums['rmse'])
    minIdx = rmseNums.index[rmseNums['rmse'] == minRMSE]
    rmseMins = pd.concat([rmseMins, rmseNums.loc[minIdx]], ignore_index=True)

In [ ]:
rmseMins.head()

In [ ]:
minimum = min(rmseMins['rmse'])
minIdx = rmseMins.index[rmseMins['rmse'] == minimum]
rmseMins.loc[minIdx]